In [1]:
#!/usr/bin/env python
# coding: utf-8

# In[1]:


import numpy as np
import matplotlib.pyplot as plt
import math
import random
import h5py
import copy
from scipy.optimize import curve_fit


# In[2]:


class Sig:
    def __init__(self):
        self.number_groups = 0
        self.sig_f = []
        self.capture = []
        self.scattering = []
        self.total = []
        self.number_of_production_neutrons = []
        self.c_value = []
        self.virtual = []
        
    def __init__(self, number_groups, sig_f, sig_c, sig_scattering_matrix, number_of_production_neutrons,
                 cs_xi_table, sig_t):
        self.number_groups = number_groups
        self.sig_f = sig_f
        self.sig_c = sig_c
        self.sig_s = []
        self.sig_scattering_matrix = sig_scattering_matrix

        for i in range(0, len(self.sig_scattering_matrix)):
            self.sig_s.append(sum(self.sig_scattering_matrix[i]))
            
        self.number_of_production_neutrons = number_of_production_neutrons
        self.cs_xi_table = cs_xi_table
        self.sig_t = sig_t
        self.delta_xs = max(sig_t)/2.
        max_total_cs = max(self.sig_t)
        self.virtual = [self.delta_xs]
        
               
    def get_virtual_cs(self):
        max_total_cs = max(self.sig_t)
        self.virtual = [cross_section - max_total_cs + self.delta_xs for cross_section in self.sig_t]
        
    def return_virtual(self):
        return self.virtual
        
    def get_fission_probability(self, energy_group_idx):
        return self.sig_f[energy_group_idx] / self.sig_t[energy_group_idx]
    
    def get_capture_probability(self, energy_group_idx):
        return self.sig_c[energy_group_idx] / self.sig_t[energy_group_idx]
        
    def get_scatter_probability(self, energy_group_idx):
        return self.sig_s[energy_group_idx] / self.sig_t[energy_group_idx]
    
    def get_number_of_groups(self):
        
        return self.number_groups
        


# In[3]:


energy_groups_2 = [0, 6.25, 2 * 1E10]

class Positon:
    def __init__(self, x, y, z):
        self.x = x
        self.y = y
        self.z = z
        
class Direction:
    def __init__(self, ets, phi):
        self.tetta_x = math.sin(ets) * math.cos(phi)
        self.tetta_y = math.sin(ets) * math.sin(phi)
        self.tetta_z = math.cos(ets)  


# In[4]:


class Particle:
    def __init__(self):
        self.coordinates = Positon(0, 0, 0)
        self.direction = Direction(0, 0)
        self.energy = 0.
        self.weight = 1
        self.energy_groups = []
        self.terminated = False
        self.path = 0.
        self.last_cell = 0
        self.energy_group = 0
        
    def set_coordinates(self, x, y, z):
        self.coordinates = Positon(x, y, z)
        
    def set_direction(self, ets, phi):
        self.direction = Direction(ets, phi)
    
    def set_direction_angle(self, angle):
        self.direction = angle
        
    def set_energy_groups(self, energy_groups):
        self.energy_groups = energy_groups
        
    def get_energy_group(self): 
        
        return self.energy_group
     #   res = next(x for x, val in enumerate(self.energy_groups)
      #                            if val > self.energy)        
       # return res - 1
    
    def set_energy_group(self, energy_group):
        
        self.energy_group = energy_group       
    
    def set_terminated(self):
        self.terminated = True
        
    def is_terminated(self):
        return self.terminated
        
    def get_weight(self):
        return self.weight
    
    def set_weight(self, weight):
        self.weight = weight
    
    def set_particle_deleted(self):
        self.weight = self.weight * 0
        
    def set_particle_fission(self, additional_weight):
        self.weight = self.weight * additional_weight
        
    def set_multiplicity(self, additional_weight):
        self.weight = self.weight * additional_weight
              
    def is_particle_deleted(self):
        return self.weight < 0.0001
    
    def add_path(self, path):
        self.path += path
    
    def get_path(self):
        return self.path
    
    def set_path(self, path):
        self.path = path
        
    def print_direction(self):
        print(str(self.direction.tetta_x) + " " + str(self.direction.tetta_y) + " " + str(self.direction.tetta_z))
        
    def print_coordinates(self):
        print(str(self.coordinates.x) + "  " + str(self.coordinates.y) + " " + str(self.coordinates.z))  
        


# In[5]:


class Plane:
    def __init__(self, A, B, C, D):
        self.A = A
        self.B = B
        self.C = C
        self.D = D
        
    def distance(self, particle):
        vp = (self.A * particle.direction.tetta_x + self.B * particle.direction.tetta_y + 
              self.C * particle.direction.tetta_z)
        
        
        if (abs(vp) < 1e-9):
            return -1
        
        distance = (self.D - self.A * particle.coordinates.x - self.B * particle.coordinates.y 
                    - self.C * particle.coordinates.z) / vp
        
        return distance 
    
    def get_normal(self):
        sq = math.sqrt(self.A * self.A + self.B * self.B + self.C * self.C)
        a_n = self.A / sq
        b_n = self.B / sq
        c_n = self.C / sq
        return [a_n, b_n, c_n]
    

    def get_sign(self, particle):
      
        sign = (self.A * particle.coordinates.x + self.B * particle.coordinates.y +
        self.C * particle.coordinates.z - self.D)
        
        if sign == 0:
            return sign    
            
        if sign < 0:
            return -1
        else:
            return 1
        
    def get_xml(self):
        
        obj_xml = " Plane with A "+ str(self.A) + " B "+ str(self.B) + " C "+ str(self.C) + " D "+ str(self.D)
        
        return obj_xml
    


# In[6]:


class Universe:
    def __init__(self, cells, xs):
        self.cells = cells
        self.xs = xs
        self.delta_xs = 0.
        
    def calculate_delta_xs(self):
        
        delta_xs = []
        for i in range(0, self.xs[0].number_groups):
            energy_group_maximum = -1
            for xs in self.xs:
                energy_group_maximum = max(energy_group_maximum, xs.sig_t[i])
            delta_xs.append(energy_group_maximum)
            
        self.delta_xs = delta_xs                          
            
    def get_delta_xs(self, energy_group):
        
        return self.delta_xs[energy_group]
    
    def get_xs_by_coordinates(self, particle):
        
        for i in range(0, len(cells)):
            cell = cells[i]
            if cell.is_inside(particle):
                particle.last_cell = i
                return self.xs[i]
        return []
    
    def is_inside(self, particle):
        
        for cell in self.cells:
            if cell.is_inside(particle):
                return True
        return False
        
    
    def is_boudary_reflective(self, particle):
        
        minumum = np.inf
        idx_of_min = 0.
        for i in range(0, len(self.cells)):
            distance, surface_idx = self.cells[i].get_minimum_distance(particle)
            if abs(distance) < minumum:
                minumum = distance
                idx_of_min = i
        if self.cells[idx_of_min].is_boudary_reflective(particle):
            return True
        else:
            return False
        
    def reflect_particle(self,  particle):
        self.cells[particle.last_cell].reflect_particle(particle)       


# In[7]:


def set_random_direction(particle):
    r = 1.0
    r1 = 1.0
    while (r**2 + r1**2 > 1.0):
        r = 2. * random.uniform(0, 1) - 1.
        r1 = 2. * random.uniform(0, 1) - 1.
        
    direction_x = 2.0 * r**2 + 2. * r1**2 - 1.0  
    particle.direction.tetta_x = direction_x
    particle.direction.tetta_y = r * math.sqrt((1.0 - direction_x**2)/(r**2+r1**2))
    particle.direction.tetta_z = r1 * math.sqrt((1.0 - direction_x**2)/(r**2+r1**2))


# In[8]:


import random

def make_initial_sources(number_of_paricles, box_size, zero_point, energy=10.0e6):
    
    step_x = box_size[0]/number_of_paricles
    step_y = box_size[1]/number_of_paricles
    step_z = box_size[2]/number_of_paricles
    
    x_coord = zero_point[0]
    y_coord = zero_point[1]
    z_coord = zero_point[2]
    
    sources = []
    for i in range(0, number_of_paricles):
        for j in range(0, number_of_paricles):
            for k in range(0, number_of_paricles):
                current_particle = Particle()
                current_particle.set_coordinates(x_coord + k * step_x, y_coord + j * step_y, z_coord+ i * step_z)
                set_random_direction(current_particle)
                energy_group = random.choice([0, 1])
                current_particle.set_energy_group(energy_group)
                current_particle.energy = energy
                current_particle.set_energy_groups(energy_groups_2)
                sources.append(current_particle)

    return sources


# In[9]:


def get_free_path(particle, delta_xs):
    
    energy_group_idx = particle.get_energy_group()
    sig_t = delta_xs
    free_path = -math.log(random.uniform(0, 1)) / sig_t
    
    return free_path  


# In[ ]:





# In[10]:


class Cell:
    def __init__(self):
        self.surfaces = []
        self.boundaries_type = []
        self.signs = []
        self.size = 0
        
    def set_boundaries_type(self, boundaries_type):
        self.boundaries_type = boundaries_type        
        
    def set_box_sizes(self, xm_size, ym_size, zm_size, x_size, y_size, z_size):
        self.xm_size = xm_size
        self.ym_size = ym_size
        self.zm_size = zm_size
        
        self.x_size = x_size
        self.y_size = y_size
        self.z_size = z_size
        
    def set_zero_point(self, x_0, y_0, z_0):
        self.x_0 = x_0
        self.y_0 = y_0
        self.z_0 = z_0
        
    def __init__(self, surfaces, signs):
        self.surfaces = surfaces
        self.signs = signs
        self.size = len(self.signs)
           
    def is_inside(self, particle):
        
        is_inside = True
        for i in range(0, self.size):
            sign = self.surfaces[i].get_sign(particle)
            if self.signs[i] != sign:
                is_inside = False
                
        return is_inside
    
    def is_boudary_reflective(self, particle):
        for i in range(0, self.size):
            sign = self.surfaces[i].get_sign(particle)
            if self.signs[i] != sign:
                if self.boundaries_type[i] == 'reflective':
                    return True
                else:
                    return False
        
    
    def get_minimum_distance(self, particle):
        
        minimum_distance = math.inf
        surface_idx = 0
        for i in range(0, len(self.surfaces)):
            current_distance = abs(self.surfaces[i].distance(particle))
            if current_distance > 0:
                minimum_distance = min(minimum_distance, current_distance)
                surface_idx = i
        return minimum_distance, surface_idx
    
    def get_xml(self):
        
        xml_obj = []
        for i in range(0, self.size):
            xml_obj.append(surfaces[i].get_xml())
            
        return xml_obj
    
    def reflect_particle(self, particle):
        
        if particle.coordinates.x > (self.x_size + self.x_0):
            delta = particle.coordinates.x % self.x_size
            particle.coordinates.x = self.xm_size + math.fabs(delta)
        
        if particle.coordinates.y > (self.y_size + self.y_0):
            
            delta = particle.coordinates.y % self.y_size
            particle.coordinates.y = self.ym_size + math.fabs(delta)
            
            
        if particle.coordinates.z > (self.z_size + self.z_0):
            
            delta = particle.coordinates.z % self.z_size
            particle.coordinates.z = self.zm_size + math.fabs(delta)
        
        
        if particle.coordinates.y < (self.ym_size + self.y_0):
            delta = particle.coordinates.y % self.ym_size
            particle.coordinates.y = self.y_size - math.fabs(delta)
            
            
        if particle.coordinates.x < (self.xm_size + self.x_0):
            delta = particle.coordinates.x % self.xm_size
            particle.coordinates.x = self.x_size - math.fabs(delta)
            
        if particle.coordinates.z < (self.zm_size + self.z_0):
            delta = particle.coordinates.z % self.zm_size
            particle.coordinates.z = self.z_size - math.fabs(delta)
                    


# In[11]:


class Multigroup_estimator:
    def __init__(self):
        self.groups_before = []
        self.weights_before = []
        self.groups_after = []
        self.weights_after = []
        
    def get_flux_ration_n_m(self, first, secound):
        
        number_before = 0.
        number_after = 0.
        for i in range(0, len(self.groups_before)):
            
            if self.groups_before[i] == first:
                number_before += self.weights_before[i]           
                if self.groups_after[i] == secound:
                    number_after += self.weights_after[i]
        
        flux_ratio = number_after / number_before
    
        return flux_ratio   


# In[12]:


def move_particle(particle, t):
    new_x = particle.direction.tetta_x * t + particle.coordinates.x
    new_y = particle.direction.tetta_y * t + particle.coordinates.y
    new_z = particle.direction.tetta_z * t + particle.coordinates.z
    particle.set_coordinates(new_x, new_y, new_z)


# In[13]:


def is_collision_virual(particle, c_s, delta_xs):
    
    energy_group_idx = particle.get_energy_group()
    random_number = random.uniform(0, 1)
    
    virtual_cs = delta_xs
    
    total_cs = c_s.sig_t[energy_group_idx]
    
    if total_cs / virtual_cs < random_number:
        return True
    else:
        return False


# In[14]:


def process_virtual_collision(particle, free_path):
    particle.add_path(free_path)


# In[15]:


def process_scattering(particle, c_s):
    
    energy_group_idx = particle.get_energy_group()   
    
    energy_group_probability = []
    energy_scattering_matrix = c_s.sig_scattering_matrix[energy_group_idx]
    
    majorant = sum(energy_scattering_matrix)
    for i in range(0, len(energy_scattering_matrix)):
        energy_group_probability.append(energy_scattering_matrix[i] / majorant)
        
    energy_groups_idxes = list(range(0, len(energy_scattering_matrix)))
    energy_group_idx = np.random.choice(energy_groups_idxes,  p=energy_group_probability)
    
    particle.set_energy_group(energy_group_idx)     


# In[16]:


def process_fission(particle, c_s):
    energy_group_idx = particle.get_energy_group()
    number_of_production_neutrons = c_s.number_of_production_neutrons[energy_group_idx]
    particle.set_particle_fission(number_of_production_neutrons)
    particle.set_terminated()
    energy_groups_idxes = list(range(0, c_s.number_groups))
    xi_matrix = c_s.cs_xi_table
    energy_group_idx = np.random.choice(energy_groups_idxes,  p=xi_matrix)
    particle.set_energy_group(energy_group_idx)
    


# In[17]:


def process_real_collision(particle, free_path, c_s, flux_group_estimator):
    
    energy_group_idx = particle.get_energy_group()
    number_of_production_neutrons = c_s.number_of_production_neutrons[energy_group_idx]   
    capture_probability = c_s.get_capture_probability(energy_group_idx)
    scatter_probability = c_s.get_scatter_probability(energy_group_idx)  
    fission_probability = c_s.get_fission_probability(energy_group_idx)  
    
    type_collision = np.random.choice(['capture', 'scatter', 'fission'], p=[capture_probability, scatter_probability,
                                                                            fission_probability])
    if type_collision == 'capture':
        particle.set_terminated()
        particle.set_weight(0.)
        
    if type_collision == 'scatter':
        process_scattering(particle, c_s)

        
    if type_collision == 'fission':    
        process_fission(particle, c_s)
        


# In[18]:


def delete_absorpbed_paricles(particles):
    
    existing_particles = []
    for i in range(0, len(particles)):
        particle = particles[i]
        if particle.get_weight() > 0.000001:
            existing_particles.append(particle)
                     
    return existing_particles


# In[22]:


def process_one_particle_history(particle, universe, estimators, flux_estimator):
    
    sum_collisions = 0.
    universe.calculate_delta_xs()
    
    while not particle.is_terminated():
        
        energy_group_before = particle.get_energy_group()
        weight_before = particle.get_weight()
        

        current_xs = universe.get_xs_by_coordinates(particle)
        energy_group_idx = particle.get_energy_group()
        delta_xs = universe.get_delta_xs(energy_group_idx)
        free_path = get_free_path(particle, delta_xs)        
        move_particle(particle, free_path)
        current_xs = universe.get_xs_by_coordinates(particle)
        
        flux_estimator.groups_before.append(particle.get_energy_group())
        flux_estimator.weights_before.append(particle.get_weight())
        
        if not universe.is_inside(particle):
            if universe.is_boudary_reflective(particle):
                universe.reflect_particle(particle)
                current_xs = universe.get_xs_by_coordinates(particle)

            else:
                particle.set_terminated()
                particle.set_weight(0.)
                return particle, sum_collisions 
        for k in estimators:
            k.add_collision(particle)
        
        if is_collision_virual(particle, current_xs, delta_xs):
            process_virtual_collision(particle, free_path)
        else:
            process_real_collision(particle, free_path, current_xs, flux_estimator)
            
    
    return particle, sum_collisions


# In[23]:


def get_weights(particles):
    
    weights = []
    for i in range(0, len(particles)):
        weights.append(particles[i].weight)
        
    return weights


# In[24]:


def make_sources(particles):
    for i in range(0, len(particles)):
        particles[i].terminated = False
        set_random_direction(particles[i])       


# In[45]:


def splitting_secound_version(particles):
    
    initial_size = len(particles)
    
    for i in range(0, initial_size):
        particle = particles[i]
        current_weight = particle.get_weight()
        if current_weight > 1:
            n_value = math.floor(current_weight)
            random_value = random.uniform(0, 1)
            if current_weight - n_value >= random_value:
                n_value += 1
            if n_value > 1:
                particle.set_weight(current_weight / n_value)
                for j in range(0, n_value - 1):
                    particles.append(copy.deepcopy(particle))


# In[46]:


def normalise_weights(particles, batch_size):

    sum_weights = 0.
    
    for i in range(0, len(particles)):
        sum_weights += particles[i].get_weight()

    for i in range(0, len(particles)):
        particles[i].weight = (particles[i].weight * batch_size) / sum_weights   
        
    return particles


# In[47]:


def russian_roulette(weights_previous, particles_current):
    
    for i in range(0, len(particles_current)):
        particle = particles_current[i]
        if particle.weight > 0 and particle.weight < 0.5 and particle.weight < weights_previous[i]:
            probability_terminate = 1. - particle.weight / weights_previous[i]
            random_number = random.uniform(0, 1)
            if probability_terminate >= random_number:
                particle.set_weight(0.)
            else:
                particle.set_weight(weights_previous[i])


# In[48]:


def terminate_outside_particles(batch_particles, universe):
    
    for j in range(0, len(batch_particles)):
        particle = batch_particles[j]
        if not universe.is_inside(particle):
            print("ERROR! outside particle")


# In[49]:


def reset_estimators(estimators, weight_previos, volume, sum_collisions, c_s):
    
    sig_t = c_s.sig_t[0]

    for estimator in estimators:
        estimator.collision_sum[-1] = estimator.collision_sum[-1]
        estimator.collision_sum.append(0.)  


# In[50]:


def get_std(values):
    
    current_std = np.std(values)/np.sqrt((len(values) - 1.))
  
    return current_std   


# In[51]:


def calculate_k_effective(idx, weights, number_interations, number_inactive, initial_size, k_effective,
                         k_effective_exp, std_k_effective):
    
    keff_cycle = sum(weights) / initial_size
    if idx > number_inactive:
        k_effective.append(keff_cycle)
        
    if idx > number_inactive + 1:
    
        k_effective_exp.append(sum(k_effective) / len(k_effective))
        
        std_k_effective_current = get_std(k_effective)
            
        std_k_effective.append(std_k_effective_current)

        print(" keff_cycle , k_effective_exp, std_k_effective " + str(keff_cycle) + "   " + str(k_effective_exp[-1]) +
                                    "  "+ str(std_k_effective[-1]))


# In[52]:


class Flux_group_estimator:
    def __init__(self, group):
        self.group = group
        self.collision_sum = [0]
        
    def add_collision(self, particle):
        if particle.get_energy_group() == self.group:
            self.collision_sum[-1]  += particle.get_weight()    


# In[53]:


import statistics
import time


def simulation_black_boundaries(universe, number_interations, number_inactive, number_of_particles, c_s, estimators,
                                volume=1):
    
    random.seed(time.time())
    k_effective = []
    k_effective_std = []
    k_effective_exp = []
    flux = []
    flux_exp = []
    num1 = random.randint(0,9)
    flux_group_estimators = []
    print(" num1  " + str(num1))
    
    box_size = [1., 1., 1.]
    zero_point = [0.1, 0.1, 0.1]
    initial_sources = make_initial_sources(number_of_particles, box_size, zero_point, energy=10.0e6)
    initial_size = len(initial_sources)
    
    weights_previous = [1.] * len(initial_sources)
    number_of_groups = universe.xs[0].get_number_of_groups()
    for i in range(0, number_interations):
        print("i == " + str(i))
        make_sources(initial_sources)
        batch_size = len(initial_sources)
        
        batch_particles = []
        flux_group_estimator = Multigroup_estimator()
        for j in range(0, batch_size):
            particle = initial_sources[j]           
            terminate_particle, sum_collisions = process_one_particle_history(particle, universe, 
                                                                            estimators, flux_group_estimator)
            batch_particles.append(terminate_particle)
            
            
        
        russian_roulette(weights_previous, batch_particles)
        
        batch_particles = delete_absorpbed_paricles(batch_particles)
        
        splitting_secound_version(batch_particles)
        
        weights_cycle = get_weights(batch_particles)
        
        calculate_k_effective(i, weights_cycle, number_interations, number_inactive, initial_size, k_effective,
                         k_effective_exp, k_effective_std)
        
        flux_group_estimators.append(flux_group_estimator)
        reset_estimators(estimators, weights_previous, volume, sum_collisions, c_s)              

        batch_particles = normalise_weights(batch_particles, initial_size)  

        initial_sources = batch_particles
        weights_previous = get_weights(batch_particles)
     
    return k_effective, k_effective_exp, k_effective_std, estimators, flux_group_estimators


# In[54]:


cs_fission_pu_239 = [0.06912, 0.06192]

cs_capture_pu_239 = [0.01344, 0.00384]

cs_scattering_pu_239 = [[0.26304, 0.0], [0.0720, 0.07824]]

cs_total_pu_239 = [0.3456, 0.2160]

cs_production_neutrons_pu_239 = [2.5, 2.70]

cs_xi_table_pu_239 = [0.425, 0.575]


pu_239 = Sig(2, cs_fission_pu_239, cs_capture_pu_239, cs_scattering_pu_239, 
                       cs_production_neutrons_pu_239,  cs_xi_table_pu_239, cs_total_pu_239)


energy_groups_2 = [0, 2 * 1E10]


# In[55]:


x_5 = Plane(1, 0, 0, 1000)
x_m5 = Plane(1, 0, 0, -1000)

y_5 = Plane(0, 1, 0, 1000)

y_m5 = Plane(0, 1, 0, -1000)


z_5 = Plane(0, 0, 1, 1000)
z_m5 = Plane(0, 0, 1, -1000)

surfaces = [x_5, x_m5, y_5, y_m5, z_5, z_m5]

signs = [-1, +1, -1, +1, -1, +1]
slab = Cell(surfaces, signs)

boundaries_type = ["reflective", "reflective", "reflective", "reflective", "reflective", "reflective"]


slab.set_boundaries_type(boundaries_type)
slab.set_box_sizes(-1000, -1000., -1000, 1000, 1000., 1000 )
slab.set_zero_point(0., 0., 0.)


# In[56]:


cells = [slab]


# In[57]:


estimators = []


# In[58]:


materials = [pu_239]


# In[59]:


U235_Reactor = Universe(cells, materials)


# In[60]:


zero_group_estimator = Flux_group_estimator(0)
first_group_estimator = Flux_group_estimator(1)


# In[61]:


estimators = [zero_group_estimator, first_group_estimator]


# In[62]:


test_number_of_particles = 20
test_number_interations = 1000
test_number_inactive = 200


# In[63]:


k_effective,k_effective_exp, k_effective_std, estimators_result, flux_group_result = simulation_black_boundaries(U235_Reactor,  
                           test_number_interations,  test_number_inactive, 
                          test_number_of_particles, pu_239, estimators)


# In[308]:


first_group_flux = []
secound_group_flux = []

number_particles = 1000
cs_total_pu_239 = [0.3360, 0.2208]

for i in range(0, len(estimators_result[0].collision_sum) - 1):
    first_group_flux.append(estimators_result[0].collision_sum[i]/(cs_total_pu_239[0]))

for i in range(0, len(estimators_result[1].collision_sum) - 1):
    secound_group_flux.append(estimators_result[1].collision_sum[i]/(cs_total_pu_239[1]))
    
average_flux_difference = []    
for i in range(0, len(first_group_flux)):
    average_flux_difference.append(secound_group_flux[i]/ first_group_flux[i])

std_flux = get_std(average_flux_difference)
flux_difference = sum(average_flux_difference)/len(average_flux_difference)


# In[309]:


difference = (flux_difference  - 0.675229) * 100000

print('difference flux == '+ str(difference)+ 'with std   '+ str(std_flux * 100000))


# In[310]:


difference = (2.683767  - k_effective_exp[-1]) * 100000

standart_deviation = k_effective_std[-1] * 100000

print(" difference from beachmark [pcm]  " + str(difference) + "  with standart deviation [pcm]+- " + str(standart_deviation))


# In[311]:


def save_simulation_results_to_file(k_effective, file_name):
    
    f = open(file_name, "w")
    
    for i in range(0, len(k_effective)):
        f.write(" cycle k effective "+ str(k_effective[i]) + '\n')
            
    
    f.close()


# In[312]:


file_name = "output.txt"
save_simulation_results_to_file(k_effective, file_name)


# In[313]:


def save_flux_to_file(estimators, file_name):
        
    k = 0
    for estimator in estimators:
        
        result_file_name = file_name + str(k)+".txt"
        
        f = open(result_file_name, "w")
        array_without_zeros = estimator.collision_sum
        for i in range(0, len(array_without_zeros)):
            f.write(" flux "+ str(array_without_zeros[i]) +'\n')
        k += 1
        
        f.close()


# In[314]:


file_name = "result_flux.txt"
save_flux_to_file(estimators, file_name)


# In[ ]:





 num1  6
i == 0
i == 1
i == 2
i == 3
i == 4
i == 5
i == 6
i == 7
i == 8
i == 9
i == 10
i == 11
i == 12
i == 13
i == 14
i == 15
i == 16
i == 17
i == 18
i == 19
i == 20
i == 21
i == 22
i == 23
i == 24
i == 25
i == 26
i == 27
i == 28
i == 29
i == 30
i == 31
i == 32
i == 33
i == 34
i == 35
i == 36
i == 37
i == 38
i == 39
i == 40
i == 41
i == 42
i == 43
i == 44
i == 45
i == 46
i == 47
i == 48
i == 49
i == 50
i == 51
i == 52
i == 53
i == 54
i == 55
i == 56
i == 57
i == 58
i == 59
i == 60
i == 61
i == 62
i == 63
i == 64
i == 65
i == 66
i == 67
i == 68
i == 69
i == 70
i == 71
i == 72
i == 73
i == 74
i == 75
i == 76
i == 77
i == 78
i == 79
i == 80
i == 81
i == 82
i == 83
i == 84
i == 85
i == 86
i == 87
i == 88
i == 89
i == 90
i == 91
i == 92
i == 93
i == 94
i == 95
i == 96
i == 97
i == 98
i == 99
i == 100
i == 101
i == 102
i == 103
i == 104
i == 105
i == 106
i == 107
i == 108
i == 109
i == 110
i == 111
i == 112
i == 113
i == 114
i == 115
i == 116
i == 117
i == 118
i == 119
i == 120
i == 121
i =

 keff_cycle , k_effective_exp, std_k_effective 2.2231795306725286   2.2162268578698834  0.0009940552064109586
i == 258
 keff_cycle , k_effective_exp, std_k_effective 2.213509546645732   2.216180007676363  0.0009778889009132272
i == 259
 keff_cycle , k_effective_exp, std_k_effective 2.2131496808386593   2.2161286462045378  0.0009625429252208122
i == 260
 keff_cycle , k_effective_exp, std_k_effective 2.2136696391543627   2.2160876627537016  0.0009472515839902288
i == 261
 keff_cycle , k_effective_exp, std_k_effective 2.215999217753756   2.2160862128356698  0.000931594582871334
i == 262
 keff_cycle , k_effective_exp, std_k_effective 2.2167646581108515   2.216097155501399  0.0009165110210869161
i == 263
 keff_cycle , k_effective_exp, std_k_effective 2.20975565145002   2.2159964967069326  0.0009074459860712133
i == 264
 keff_cycle , k_effective_exp, std_k_effective 2.218083963697884   2.2160291133786663  0.0008937499625793488
i == 265
 keff_cycle , k_effective_exp, std_k_effective 2.2305240

 keff_cycle , k_effective_exp, std_k_effective 2.2238345629124026   2.215936691996281  0.0005903698472081011
i == 328
 keff_cycle , k_effective_exp, std_k_effective 2.1959267460073746   2.2157803642932428  0.000606241720384338
i == 329
 keff_cycle , k_effective_exp, std_k_effective 2.213854287500649   2.215765433465393  0.0006017090898091455
i == 330
 keff_cycle , k_effective_exp, std_k_effective 2.217811972038232   2.2157811760697994  0.0005972701226819106
i == 331
 keff_cycle , k_effective_exp, std_k_effective 2.2165557601459054   2.2157870889253424  0.0005927227660113025
i == 332
 keff_cycle , k_effective_exp, std_k_effective 2.2122203454483387   2.215760068141426  0.0005888356015105471
i == 333
 keff_cycle , k_effective_exp, std_k_effective 2.2203458062398407   2.215794547375248  0.0005854077534708804
i == 334
 keff_cycle , k_effective_exp, std_k_effective 2.2145240122782854   2.215785065770047  0.0005810999740901087
i == 335
 keff_cycle , k_effective_exp, std_k_effective 2.2183801

 keff_cycle , k_effective_exp, std_k_effective 2.2195537398792196   2.2160179574617103  0.0004967670210173587
i == 398
 keff_cycle , k_effective_exp, std_k_effective 2.221525614119371   2.2160457739094763  0.0004950338636776084
i == 399
 keff_cycle , k_effective_exp, std_k_effective 2.2110228430485535   2.216020533050879  0.0004931863008738643
i == 400
 keff_cycle , k_effective_exp, std_k_effective 2.218844236717275   2.216034651569211  0.0004909172360549586
i == 401
 keff_cycle , k_effective_exp, std_k_effective 2.224475095960976   2.2160766438298665  0.0004902704104070482
i == 402
 keff_cycle , k_effective_exp, std_k_effective 2.2124270985040515   2.2160585767737984  0.0004881717337216238
i == 403
 keff_cycle , k_effective_exp, std_k_effective 2.1956542719371477   2.2159580629568687  0.0004960511778552996
i == 404
 keff_cycle , k_effective_exp, std_k_effective 2.2082137168999916   2.215920100476198  0.0004950712086824358
i == 405
 keff_cycle , k_effective_exp, std_k_effective 2.21226

 keff_cycle , k_effective_exp, std_k_effective 2.2134418240914138   2.2162065973931835  0.0004169425232111353
i == 468
 keff_cycle , k_effective_exp, std_k_effective 2.218995098121095   2.2162170022466463  0.0004155141480584999
i == 469
 keff_cycle , k_effective_exp, std_k_effective 2.220126335001959   2.2162315350821675  0.000414221622442273
i == 470
 keff_cycle , k_effective_exp, std_k_effective 2.2213682868063493   2.2162505600885534  0.0004131229159929915
i == 471
 keff_cycle , k_effective_exp, std_k_effective 2.2027579518180893   2.216200771866153  0.00041459600769914903
i == 472
 keff_cycle , k_effective_exp, std_k_effective 2.2077875460346603   2.2161698408888313  0.0004142253963788367
i == 473
 keff_cycle , k_effective_exp, std_k_effective 2.2118409942934396   2.216153984307896  0.0004130097991582772
i == 474
 keff_cycle , k_effective_exp, std_k_effective 2.217978490822655   2.2161606430907965  0.0004115535752628567
i == 475
 keff_cycle , k_effective_exp, std_k_effective 2.2158

...

 keff_cycle , k_effective_exp, std_k_effective 2.2260227966284925   2.2163746981905064  0.0003092914923505186
i == 678
 keff_cycle , k_effective_exp, std_k_effective 2.2148571362278426   2.216371523374685  0.00030866008895707417
i == 679
 keff_cycle , k_effective_exp, std_k_effective 2.2148470400114326   2.216368340737183  0.00030803147282997565
i == 680
 keff_cycle , k_effective_exp, std_k_effective 2.2124109445911513   2.216360096161879  0.00030749961597487184
i == 681
 keff_cycle , k_effective_exp, std_k_effective 2.214134997915099   2.216355470177998  0.0003068945246382117
i == 682
 keff_cycle , k_effective_exp, std_k_effective 2.213513749352387   2.2163495744916375  0.0003063138951420162
i == 683
 keff_cycle , k_effective_exp, std_k_effective 2.2306588359214663   2.2163792002916995  0.0003071113280380998
i == 684
 keff_cycle , k_effective_exp, std_k_effective 2.2229956739016448   2.21639287069172  0.0003067808769411034
i == 685
 keff_cycle , k_effective_exp, std_k_effective 2.2192

 keff_cycle , k_effective_exp, std_k_effective 2.2163501776931698   2.2163207120106856  0.00029020168407793944
i == 748
 keff_cycle , k_effective_exp, std_k_effective 2.2251150873232937   2.216336760140818  0.0002901158364255517
i == 749
 keff_cycle , k_effective_exp, std_k_effective 2.2147795595370194   2.2163339237098456  0.0002896008008525342
i == 750
 keff_cycle , k_effective_exp, std_k_effective 2.1981697113871834   2.216300897869259  0.00029095421149532847
i == 751
 keff_cycle , k_effective_exp, std_k_effective 2.211931419491749   2.216292967781459  0.00029053392943028826
i == 752
 keff_cycle , k_effective_exp, std_k_effective 2.2110852710792472   2.2162835335483035  0.0002901605343633178
i == 753
 keff_cycle , k_effective_exp, std_k_effective 2.2186584598747507   2.2162878281709553  0.00028966719440719956
i == 754
 keff_cycle , k_effective_exp, std_k_effective 2.213601369251018   2.2162829789671283  0.00028918451658861114
i == 755
 keff_cycle , k_effective_exp, std_k_effective 2

 keff_cycle , k_effective_exp, std_k_effective 2.216025142669463   2.2163862590801453  0.0002729387011448699
i == 818
 keff_cycle , k_effective_exp, std_k_effective 2.214861044684493   2.2163837910956863  0.0002725078708175282
i == 819
 keff_cycle , k_effective_exp, std_k_effective 2.208886186715555   2.2163716786491916  0.0002723367655941785
i == 820
 keff_cycle , k_effective_exp, std_k_effective 2.2100826150557342   2.216361534998234  0.00027208630643488983
i == 821
 keff_cycle , k_effective_exp, std_k_effective 2.2214693452412515   2.216369760135502  0.0002717723055177974
i == 822
 keff_cycle , k_effective_exp, std_k_effective 2.2162774742804228   2.2163696117659595  0.0002713350613178199
i == 823
 keff_cycle , k_effective_exp, std_k_effective 2.210540925177812   2.2163602559287394  0.00027106069096055434
i == 824
 keff_cycle , k_effective_exp, std_k_effective 2.2171341278286985   2.2163614961080667  0.0002706287918263052
i == 825
 keff_cycle , k_effective_exp, std_k_effective 2.216

 keff_cycle , k_effective_exp, std_k_effective 2.213306378195668   2.2165021767013884  0.00025469792903748607
i == 888
 keff_cycle , k_effective_exp, std_k_effective 2.205238238185356   2.2164858046977316  0.00025485387768180205
i == 889
 keff_cycle , k_effective_exp, std_k_effective 2.223543332413475   2.216496047843908  0.00025468978278929573
i == 890
 keff_cycle , k_effective_exp, std_k_effective 2.202220990829533   2.216475359355482  0.00025516049665414586
i == 891
 keff_cycle , k_effective_exp, std_k_effective 2.2167009394377137   2.2164756858100145  0.0002547911755515638
i == 892
 keff_cycle , k_effective_exp, std_k_effective 2.209354894902627   2.216465395649744  0.0002546307222039871
i == 893
 keff_cycle , k_effective_exp, std_k_effective 2.218471641403412   2.216468290665261  0.00025427950487508043
i == 894
 keff_cycle , k_effective_exp, std_k_effective 2.2192896619196474   2.2164723560417086  0.0002539453866470038
i == 895
 keff_cycle , k_effective_exp, std_k_effective 2.2209

 keff_cycle , k_effective_exp, std_k_effective 2.2120122986469126   2.2164654318583  0.00024173010608603257
i == 958
 keff_cycle , k_effective_exp, std_k_effective 2.2139234228037474   2.2164620782843496  0.00024143428234946895
i == 959
 keff_cycle , k_effective_exp, std_k_effective 2.2242901782158127   2.2164723919864993  0.0002413364600650211
i == 960
 keff_cycle , k_effective_exp, std_k_effective 2.2119999325872834   2.2164665071715004  0.0002410905352694435
i == 961
 keff_cycle , k_effective_exp, std_k_effective 2.2110775551418227   2.2164594257627885  0.00024087763272821828
i == 962
 keff_cycle , k_effective_exp, std_k_effective 2.2164567658243857   2.2164594222720555  0.00024056131267915988
i == 963
 keff_cycle , k_effective_exp, std_k_effective 2.2093047325149384   2.21645004522126  0.0002404287508034748
i == 964
 keff_cycle , k_effective_exp, std_k_effective 2.2108173839413285   2.2164426726279616  0.00024022700676414326
i == 965
 keff_cycle , k_effective_exp, std_k_effective 2